In [3]:
!pip install git+https://github.com/openai/shap-e.git

  Cloning https://github.com/openai/shap-e.git to /tmp/pip-req-build-awcrbx8n
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e.git /tmp/pip-req-build-awcrbx8n
  Resolved https://github.com/openai/shap-e.git to commit 50131012ee11c9d2617f3886c10f000d3c7a3b43
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-4oe2eidk/clip_7a2dac96f0704f75842485296eec49d0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-4oe2eidk/clip_7a2dac96f0704f75842485296eec49d0
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [17]:
import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh
from PIL import Image
import os

def load_image(image_path):
    """Load and preprocess an image for SHAP-E."""
    image = Image.open(image_path).convert('RGB')
    image = image.resize((224, 224))  # Resize to expected input size
    return image

def generate_3d_model(input_type, input_data, output_path, device):
    """Generate a 3D model from text or image input and save it."""
    # Load SHAP-E models
    xm = load_model('transmitter', device=device)
    model = load_model('text300M' if input_type == 'text' else 'image300M', device=device)
    diffusion = diffusion_from_config(load_config('diffusion'))

    # Prepare input
    batch_size = 1
    guidance_scale = 15.0
    if input_type == 'text':
        prompt = input_data
    else:
        prompt = load_image(input_data)

    # Sample latents and generate 3D model
    latents = sample_latents(
        batch_size=batch_size,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[prompt] if input_type == 'text' else dict(images=[prompt])),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    # Decode latent to mesh and save
    for i, latent in enumerate(latents):
        t = decode_latent_mesh(xm, latent).tri_mesh()
        with open(output_path, 'wb') as f:
            t.write_ply(f)

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Choose input type: (1) Text prompt, (2) Image")
    choice = input("Enter 1 or 2: ").strip()

    if choice == '1':
        input_type = 'text'
        input_data = input("Enter text prompt: ").strip()
        output_filename = 'model_from_text.ply'
    elif choice == '2':
        input_type = 'image'
        input_data = input("Enter image file path: ").strip()
        if not os.path.exists(input_data):
            print("Image file does not exist!")
            return
        output_filename = 'model_from_image.ply'
    else:
        print("Invalid choice!")
        return

    output_path = os.path.join(os.getcwd(), output_filename)
    print(f"Generating 3D model... Output will be saved to {output_path}")
    generate_3d_model(input_type, input_data, output_path, device)
    print(f"3D model saved to {output_path}")

if __name__ == "__main__":
    main()
    

Choose input type: (1) Text prompt, (2) Image


Enter 1 or 2:  1
Enter text prompt:  a red chair


Generating 3D model... Output will be saved to /kaggle/working/model_from_text.ply


  0%|          | 0/64 [00:00<?, ?it/s]

3D model saved to /kaggle/working/model_from_text.ply


In [18]:
# Install trimesh (for 3D model conversion)
!pip install -q trimesh

import trimesh

# Load your PLY model
ply_path = '/kaggle/working/model_from_text.ply'
mesh = trimesh.load(ply_path)

# Export to OBJ and STL
obj_path = '/kaggle/working/model_from_text.obj'
stl_path = '/kaggle/working/model_from_text.stl'

mesh.export(obj_path)
mesh.export(stl_path)

print(f"Saved OBJ to {obj_path}")
print(f"Saved STL to {stl_path}")


Saved OBJ to /kaggle/working/model_from_text.obj
Saved STL to /kaggle/working/model_from_text.stl
